In [4]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence as PACK, pad_packed_sequence as PAD
from torch.nn.utils.rnn import pad_sequence
from abc import ABC, abstractmethod
import shutil
import nltk

ModuleNotFoundError: No module named 'torch'

In [4]:
# !rm -rf ./text-segmentation
!git clone https://github.com/koomri/text-segmentation.git
!rm -rf ./data
!mkdir -p ./data/choi
!cp -r ./text-segmentation/data/choi ./data/

Cloning into 'text-segmentation'...
Updating files:  54% (528/973)
Updating files:  55% (536/973)
Updating files:  56% (545/973)
Updating files:  57% (555/973)
Updating files:  58% (565/973)
Updating files:  59% (575/973)
Updating files:  60% (584/973)
Updating files:  61% (594/973)
Updating files:  62% (604/973)
Updating files:  63% (613/973)
Updating files:  64% (623/973)
Updating files:  65% (633/973)
Updating files:  66% (643/973)
Updating files:  67% (652/973)
Updating files:  68% (662/973)
Updating files:  69% (672/973)
Updating files:  70% (682/973)
Updating files:  71% (691/973)
Updating files:  72% (701/973)
Updating files:  73% (711/973)
Updating files:  74% (721/973)
Updating files:  75% (730/973)
Updating files:  76% (740/973)
Updating files:  77% (750/973)
Updating files:  78% (759/973)
Updating files:  79% (769/973)
Updating files:  80% (779/973)
Updating files:  81% (789/973)
Updating files:  82% (798/973)
Updating files:  83% (808/973)
Updating files:  84% (818/973)
Upd

In [2]:
from transformers import AutoTokenizer, AutoModel

# SMALL SBERT
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
nse_model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

c:\Users\brazen\Miniconda3\envs\default\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def batch_calc_docs_embs(batch_docs):
        list_docs_embs = []
        for doc in batch_docs:
            tokenized_docs = tokenizer(
                doc,
                padding=True,
                truncation=True,
                return_tensors='pt',
                return_token_type_ids=False,
                return_attention_mask=False
                )

            with torch.no_grad():
                tokenized_docs = {k: v.to(nse_model.device) for k, v in tokenized_docs.items()}
                model_output = nse_model(**tokenized_docs)

            docs_embs = 0
            docs_embs = model_output.last_hidden_state[:, 0, :]
            docs_embs = torch.nn.functional.normalize(docs_embs)
            list_docs_embs.append(docs_embs)
        batch_docs_embs = pad_sequence(list_docs_embs, batch_first=True)
        return batch_docs_embs

sample_text = """We use the Pk metric as defined in Beeferman
et al. (1999) to evaluate the performance of our
model. Pk is the probability that when passing a
sliding window of size k over sentences, the sentences at the boundaries of the window will be incorrectly classified as belonging to the same segment (or vice versa). To match the setup of Chen
et al. (2009), we also provide the Pk metric for a
sliding window over words when evaluating on the
datasets from their paper"""
sent_detector = nltk.data.load('tokenizers/punkt/russian.pickle')
sample_sents = sent_detector.tokenize(sample_text)
sample_sents = [sample_sents]
sample_lengths = [len(s) for s in sample_sents]
sample_lengths = torch.LongTensor(sample_lengths)
sample_embs = batch_calc_docs_embs(sample_sents)
sample_targets = torch.zeros(1, len(sample_sents[0]))
sample_targets[:, sample_targets.shape[1]//2] = 1 # split into two segments in the middle

# create dummy batch of copies
batch_size = 2
sample_embs = sample_embs.expand(batch_size, -1, -1)
sample_targets = sample_targets.expand(batch_size, -1)
sample_lengths = sample_lengths.expand(batch_size)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [4]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
def delete_experiment(exp_name):
    if os.path.exists(exp_name):
        ans = input(f"Experiment '{exp_name}' already exists. Delete [y/n]?")
        if ans.lower() == 'y':
            !rm -rf {exp_name}
    else:
        print(f"Experiment doesn't exist.")

In [3]:
import shutil

filepath = './train_fit.py'
! chmod 755 {filepath}
encoder_full_name = "cointegrated/rubert-tiny2"
dataset_name = 'lenta'
# exp_name = dataset_name + 
# exp_name = 'lenta_full'
exp_name = 'lenta_dropout_search'
delete_experiment(exp_name)

! python {filepath} \
    --verbose \
    --dataset {dataset_name} --corus \
    -exp {exp_name} \
    --wandb --wandb_key aee284a72205e2d6787bd3ce266c5b9aefefa42c \
    --metric='F1' \
    -arc='BiLSTM' \
    --encoder={encoder_full_name} \
    --hidden_units=256 --num_layers=2 \
    --dropout_in 0.2 --dropout_out=0.1 \
    -lr=0.001 -bs=32 \
    --patience=5 \
    --threshold=0.5 \
    --max_epochs=2 \
    --max_docs_cnt=10 \
    --hyperparameters_search \
    -huss=256 -nlss=2 \
   -diss 0.3 -doss 0.7 0.1\
    # --save_embeddings \
#     --max_docs_frac=0.25 \
#     --max_epochs=100 \
 ##     --embeddings_directory '../embeddings'
    # --online_encoding
#     --embeddings_directory='embeddings/'
    



2024-06-12 17:43:29.240219: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-06-12 17:43:29.240320: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/brazen/miniconda3/envs/ds-3.9/lib/python3.9/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
[nltk_data] Downloading package punkt to /home/brazen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /home/brazen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /home/brazen/nltk_data...
[

In [11]:
import wandb
key='aee284a72205e2d6787bd3ce266c5b9aefefa42c'
wandb.login(key=key)

True

In [15]:
import os
import numpy as np

file_num = 0
embeddings = [0] * 4
path = '/home/brazen/NSE-TopicSegmentation/embeddings/lenta/cointegrated/rubert-tiny2_train/embeddings_0.npy'
embeddings[int(file_num)] = np.load(path)

In [16]:
embeddings[0].shape

(229, 312)

In [9]:
import subprocess

url = 'https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz'
cmd = ['curl', '-O', url]
out = subprocess.run(
    cmd, text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
if out.returncode != 0:
    print("Error while loading file:")
    print(out.stdout)